<a href="https://colab.research.google.com/github/Wong-Vee-Kin/MWSD/blob/master/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>FEATURE REPRESENTATION FOR MALAY WORD SENSE DISAMBIGUATION</h1>


This notebook contains the function cluster_sentences to score Malay sentence K-means clustering results based on different feature representation methods. The goal is to identify feature representation methods that gives the best ARI score for sense disambiguation.

The inputs to this function are the sentence list, target word and ground truths.

The input sentences are transformed into stemmed version and parts-of-speech (pos) versions of sentences, as in the diagram below. The goal is to determine if any of these forms of representation are better for sense disambiguation.

![](https://drive.google.com/uc?id=1k8DbjD20fVZPjFs4c21H-hFuIQGKfzkp)

The input can be represented in matrix form as below:

>>>>>>$Sentences = \begin{bmatrix}
c_{11} & ... & c_{1v}\\ 
\vdots  & \ddots  & \vdots \\ 
c_{n1} & ... & c_{nv} 
\end{bmatrix}\begin{bmatrix}
word_{1}\\ 
\vdots \\ 
word_{v}
\end{bmatrix}$

In many text processing tasks, normalization is done at the corpus and document level. For this task, we can consider the entire collection of input sentences as the corpus and the individual sentences as the document. Each of the coefficient $c_{nv}$ in the matrix can be represented as term count, term freq and tfidf which are then defined as follows:

>>>>>>$term \,count, tc = \sum_{L}w$

>>>>>$term \,frequency, tf = \sum_{L}w/L$

>>$term \,frequency-inverse \,document \,frequency, tc = \sum_{L}w*\frac{\sum{D}}{D_{w}}$

$w = word$

$L = number \,of \,words \,in \,the \,sentence$

$D = document \,i.e. \,sentences$

$D_{w} = document \,that \,contains \,the \,word \,"w"$


Besides representing the words in original form, stemmed and parts of speech, the algorithm also tests clustering effectiveness by varying the collocation window i.e. filtering only words that are neighboring the target word, as depicted in the picture below:

![](https://drive.google.com/uc?id=1nvBm3vcmPeRIqb_OZMshdl_aMmpXy3jP)


The window can be either left (including only words preceding the target word), right or both.

Scoring of the clustering result is based on the adjusted-Rand-index, describe as follows:

>>>>$Adjusted \,Rand \,Index, ARI = \frac{RI-E[RI]}{max(RI)-E[RI]}$

>>>>>>$Rand \,Index, RI = \frac{a+b}{C_{2}^{n}}$

The notebook contains sample sentences with targetword 'madu' which is a common Malay polysemy. Other sentence lists can be uploaded into this notebook either by pickles or by creating a new code cell with lists defined explicitly.

In [0]:
#@title Mount disk
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pathlib
from os import listdir
for file in listdir('gdrive/My Drive/Colab notebooks/clustering data'):
  print('gdrive/My Drive/Colab notebooks/clustering data'+'/'+file)

gdrive/My Drive/Colab notebooks/clustering data/outputs
gdrive/My Drive/Colab notebooks/clustering data/cetek.pkl
gdrive/My Drive/Colab notebooks/clustering data/jalan.pkl
gdrive/My Drive/Colab notebooks/clustering data/bela_0.pkl
gdrive/My Drive/Colab notebooks/clustering data/bela_2.pkl
gdrive/My Drive/Colab notebooks/clustering data/bela_1.pkl
gdrive/My Drive/Colab notebooks/clustering data/buah_1.pkl
gdrive/My Drive/Colab notebooks/clustering data/gantung.pkl
gdrive/My Drive/Colab notebooks/clustering data/keras.pkl
gdrive/My Drive/Colab notebooks/clustering data/buah_0.pkl
gdrive/My Drive/Colab notebooks/clustering data/buah_2.pkl
gdrive/My Drive/Colab notebooks/clustering data/masak.pkl
gdrive/My Drive/Colab notebooks/clustering data/madu_0.pkl
gdrive/My Drive/Colab notebooks/clustering data/bekas.pkl
gdrive/My Drive/Colab notebooks/clustering data/semak.pkl
gdrive/My Drive/Colab notebooks/clustering data/madu_2.pkl
gdrive/My Drive/Colab notebooks/clustering data/tampar.pkl
gdriv

First, we initiate all the modules and functions that are used in this notebook

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import k_means
from sklearn import metrics
import re
import numpy as np
import pickle

def term_freq(X):
    #convert countvectorizer output to normalized freq
    X=X.toarray()
    calcrecip=[]
    #mask off rows that are all 0s to avoid divide by 0
    for i in  range(0,len(X)):
      if sum(X[i])==0:
        calcrecip.append(False)
      else:
        calcrecip.append(True)
    calcrecip=np.array(calcrecip)
    fnormalizer=np.reciprocal(np.sum(X, axis=1, dtype=np.float64), where=calcrecip)
    y=np.multiply(X.transpose(), fnormalizer.transpose())
    yt=y.transpose()
    return yt
  
def find_target(sentence, targetword):
  val=False
  if targetword in sentence.lower().split(' '):
    val=True
  return val

Malaya (https://malaya.readthedocs.io/en/latest/) is the library used for pos tagging and stemming

In [0]:
!pip install malaya
import malaya
pos=malaya.pos.deep_model('attention')
stemmer = malaya.stem.deep_model('bahdanau')

W0713 00:06:29.952741 140131205232512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/malaya/_utils/_utils.py:42: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0713 00:06:29.954361 140131205232512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/malaya/_utils/_utils.py:43: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.

W0713 00:06:30.558299 140131205232512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/malaya/_utils/_utils.py:38: The name tf.InteractiveSession is deprecated. Please use tf.compat.v1.InteractiveSession instead.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive sessio

Sample sentences:

In [0]:
input_sentences=[' hektar dan dalam tempoh tiga tahun, pokok nangka madu tersebut berjaya mengeluarkan hasil dan dikomersialkan.', ' peratus daripada sejumlah 19,000 pasangan berbulan madu yang membeli pakej pelancongan yang ditawarkannya tahun lepas memilih pusat peranginan di Amerika Syarikat itu sebagai destinasi.', ' Tanaman unik yang dianggap emas merah dunia ini digunakan sejak turuntemurun dalam masakan dan kini semakin meluas digunakan sebagai bahan pewarna,minyak wangi dan dalam bidang perubatan serta sebagai campuran dengan madu untukmakanan kesihatan.\r', ', "Pemimpin baharu hormatilah pemimpin lama dan pemimpin lama perlu menyayangi pemimpin baharu dan prinsip habis madu sepah dibuang perlu dielakkan demi kelangsungan parti," katanya.', ', Kilang berskala perusahaan kecil dan sederhana CocoDamia di Skudai mengeluarkan 4,000 kilogram coklat sebulan, terdiri daripada Coklat IQ, Coklat Bar dan Coklat Tenaga, dengan campuran makanan sunnah seperti lubanah atau dikenali juga sebagai kemenyan Arab (resin beraroma), kismis, kurma, madu dan air zamzam bagi memenuhi permintaan pelanggan.', ', Pengarahnya, P Prem Anand Pillai berpuas hati dengan prestasi barisan pelakon, terutamanya gandingan Lisa Surihani dan Nabil Ahmad yang menghidupkan karakter pasangan suami isteri Sofia dan Faris yang berbulan madu di Paris.\r', '\\n Selamat bergelar suami isteri, pelakon veteran Datuk Jalaluddin Hassan, 63, menyatakan hasrat untuk berbulan madu bersama isterinya, Rapidah Abdul Ghani, 41, di Moscow, Rusia.\r', '?Buat masa ini saya belum ada pekerja tetapi saya cuma ajar penduduk di sini macam mana menguruskan madu kelulut ini, jadi mereka boleh tambah pendapatan,? ujarnya yang juga merupakan Pengerusi Madu Kelulut Taman Flora Height.', '?Insyaallah? rancangan berbulan madu belum ada lagi.', '?Kehidupan saya sekeluarga memang berubah selepas mengusahakan madu kelulut.', '?Madu kelulut yang kami hasilkan ini akan dibekalkan kepada Koperasi Pemungut Madu Lebah Asli Terengganu.', '?Manisan tu boleh buat cendol, boleh makan dengan kuih, boleh makan dengan ketupat, macam madu la, cuma ni manisan tebu.', '?Minuman istimewa di kafe ini pula sudah tentunya kopi ?kaw? dan terdapat juga teh tarik madu atau boleh cuba minuman bali madu lemon atau madu lemon bagi menyegarkan tubuh sebelum memulakan hari masing-masing,? katanya.', '?Pada mulanya saya mengusahakan madu kelulut di luar, kemudian saya terfikir kenapa tidak saya buat di sini untuk menaikkan nama taman ini dan tambah pendapatan penduduk di sini.', '?Saya sudah beritahu mereka bahawa tempoh bulan madu sudah tamat dan jurulatih juga faham.', '?Tangkapan pertama pada 19 November di Kelantan melibatkan seorang penjual madu berusia 28 tahun yang menerima arahan daripada ahli kumpulan Daish Syria, Akel Zainal untuk melancarkan serangan di Malaysia.', '?Saya akan mengumumkan keputusan saya selepas balik dari berbulan madu kerana pada masa itu saya akan beri fokus sepenuhnya kepada bola sepak? tambahnya.', '3) MADU Madu boleh membantu menenangkan kerongkong anda dan mengurangkan batuk.', 'Ada juga sebahagian dari rakyat yang mempertahankan keputusan dahulu kerana merasakan masa berbulan madu belum selesai.\r', 'Ada tanda-tanda bulan madu itu sudah hilang madunya, mungkin berganti pahit.', 'Ahmad Shabery berkata pengusaha madu kelulut perlu memberi penekanan terhadap penjagaan kualiti bagi mengelak pengguna tertipu dan perusahaan dapat bertahan lama.', 'Akibatnya, ramai netizen beranggapan Rita menikam belakang sahabat karibnya, Izreen Azminda yang juga merupakan bekas madu Intan Liana.', 'anoharan berkata kejadian berlaku kira-kira 7 malam itu apabila wanita berusia 35 tahun didatangi madu dan abangnya yang mengacukan pistol ke dahinya sebelum memukul muka, kepala dan badannya.', 'Antara aktiviti promosi yang akan diadakan ialah Kelulut Walk 2016 yang akan melibatkan duta penggunaan dan pengamal tegar madu lebah kelulut iaitu Pok Sabri Ali dari Terengganu.', 'Atau cuba epal yang dicelupkan dalam madu untuk snek yang enak.', 'BACA: Jalaluddin mahu berbulan madu di Moscow tapi.', 'BACA: Rita Rudaini bertemu madu Izreen Azminda BACA: Gara-gara isteri pertama Ude, jualan tudung Alif Syukri naik 50 peratus Pemuzik yang terlibat dengan program Gegar Vaganza itu memfailkan permohonan cerai itu pada Januari lalu mengikut Seksyen 47 Enakmen Undang- Undang Keluarga Islam (Negeri Selangor) 2003.', 'BACA: Sah! Azrinaz Mazhar Hakim selamat bernikah 1 Mei lalu Jelasnya, perjalanan mereka ke negara itu bukan atas niat berbulan madu tapi kemanusiaan.', 'Bagaimanapun disebabkan jadual kerja mereka, Taeyang dan Hyo Rin belum mempunyai perancangan untuk berbulan madu selepas berkahwin nanti.', 'BAGAN DATUK ": Timbalan Perdana Menteri Datuk Seri Ahmad Zahid Hamidi pada Sabtu mengingatkan pemimpin politik khususnya dari UMNO supaya tidak mengamalkan sikap habis madu sepah dibuang .', 'Bagi pasangan pengantin baharu, Isma Syukri Ishak dan isterinya, Nur Faradila Daud, kedua-duanya berusia 28 tahun berkata mereka telah lama merancang untuk berbulan madu di pulau peranginan itu selepas bernikah pada minggu lalu.', 'Bapanya, Zairulnizam Abdullah, 38, lumpuh selepas patah tulang rusuk dan pinggul selepas terjatuh dari sebatang pokok ketika mengambil madu lebah pada 2014.', 'Baru-baru ini, beberapa portal asing mendakwa terdapat siasatan sedangdijalankan terhadap produk madu yang dikatakan mempunyai kandungan beracun dantoksin yang boleh memudaratkan kesihatan.', 'Beberapa tahun kebelakangan ini juga, Kak Zack amalkan makanan sunnah seperti cuka epal yang dicampur dua camca madu dan sedikit air, selain pengambilan suplemen dan sesudu minyak zaitun setiap pagi.', 'Begitu juga, kita melihat pelancong Malaysia melawat Arab Saudi untuk berbulan madu mereka di Mekah," katanya.', 'BEKAS madu pelakon Izreen Azminda, Intan Liana Saleh mengucapkan tahniah kepada aktres terbabit yang kini sedang hamil.', 'Beliau berkata kumpulan karbohidrat yang ringkas seperti gula pasir, gula batu, gula perang, gula melaka, madu dan susu pekat manis perlu dihadkan pengambilannya.', 'Beliau kini sedang giat menjalani latihan dengan menjadikan madu lebah kelulut sebagai makanan utama menambah tenaga tanpa menggunakan makanan tambahan kesihatan yang lain dan ubat-ubatan.', 'Beliau menyifatkan "masa untuk berbulan madu sudah berakhir".', 'Berbulan madu di mana? Kami tidak terbang jauh, kerana kami percaya pada prinsip berjimat-cermat.', 'Bercadang untuk berbulan madu ke tanah suci Mekah bagi menunaikan umrah bersama isteri, angah berkata dia sememangnya bersyukur dipertemukan dengan amira selepas mula bertentang mata tiga tahun lalu.', 'Beruang madu (Helarctos malayanus) adalah spesies beruang terkecil di dunia dan diklasifikasikan sebagai spesies terancam.', 'Biarkan ia terendam seketika kemudian campurkan madu secukup rasa dan minum ketika air dalam keadaan hangat atau suam bagi melegakan sakit tekak.', 'Bila diperkenalkan madu kelulut saya terus mencubanya untuk beberapa ketika.', 'Cuma saya terus amal memakan madu dan habatus sauda dan beberapa makanan yang sesuai untuk saya," katanya.', 'Dalam pada itu, Pusat Konservasi Beruang Madu Borneo di Sepilok ditutup hari ini berikutan kawasan sekitar pusat berkenaan dilanda banjir kilat.', 'Demi cinta dan sayang kepada parti, perlulah ada dasar kepedulian jangan sudah habis madu sepah dibuang.\r', 'Eksport utama pertanian Malaysia ke Australia terdiri daripada barang makanan, kopi, koko, rempah-ratus, bijirin, gula, madu manakala import utama membabitkan daging, produk tenusu, bijirin, barang makanan dan ternakan hidup.', 'Entri-entri yang dimuat naik Intan turut mengundang tanda tanya dalam kalangan pengikutnya dan sudah tentulah ia dikaitkan dengan statusnya sebagai madu kepada pelakon Izreen Azminda.', 'Fazura dan Fattah telah selamat bernikah dalam satu majlis tertutup pada 27 November 2017 Pasangan itu kini berada di London untuk menikmati percutian bulan madu mereka.', 'Fokus pentadbiran PH tergelincir sedikit kerana masa bulan madu terlalu lama.', 'GUA MUSANG : Banjir besar pada penghujung tahun 2014 yang telah menenggelamkan tanaman nangka madu tidak memutuskan semangat seorang pesara di Taman Wangi, Gua Musang, Kelantan untuk terus berjaya.', 'Hasil sedutan madu kelulut itu kemudiannya dipasarkan oleh penduduk.', 'Ibrahim dan Misbah memasang cita-cita untuk memperluaskan lagi pasaran madu kelulut serta menambah hasil pengeluaran sedia ada pada masa hadapan.', 'Ini diakui sendiri oleh Ibrahim Mustapha, 46 tahun, dari Kampung Banggol Nyior, yang kini mula beralih arah daripada hanya menjadi penoreh getah kepada menjadi pengusaha ternakan madu kelulut.', 'Interpark Tour, satu lagi agensi pelancongan terkenal, turut menyatakan bahawa Hawaii adalah lokasiberbulan madu paling popular tahun lepas diikuti Phuket, Maldives, Bali, Cancun dan Danang di Vietnam.', 'Izhar yang turut mengusahakan tanaman kelapa dara dan madu kelulut, berkata idea untuk menghasilkan pasu daripada tuala tercetus selepas menonton video mengenai pembuatan pasu dengan kaedah sama menerusi Internet beberapa bulan lalu.', 'Janji madu beracun yang tidak mungkin dapat dilaksanakan.', 'Jika sebelum ini pendapatannya hanya sekitar RM500 sebulan hasil upah menoreh getah, kini ia mencecah hampir RM2,000 hasil penjualan madu kelulut.', 'Kali kedua mereka memasang jerat pada 1 April, mereka mendapati dua ekor beruang madu terjerat.', 'KANOWIT : Industri pengeluaran madu lebah kelulut di Sarawak berpotensi besar untuk dimajukan berikutan negeri ini masih lagi kaya dengan khazanah alamnya.', 'KANOWIT : Pejalan kaki solo dan pengamal madu lebah kelulut, Pok Sabri Ali dari Terengganu akan melakukan misi berjalan kaki secara solo ke seluruh negara sejauh 2,000 kilometer (km) untuk dicatatkan dalam Malaysia Book of Records (MBR).', 'Kata Ray lagi, mengapa Hafiz perlu ragu-ragu dengan perkahwinannya sedangkan dia dan Joy berbulan madu sambil menjayakan program terbitan syarikat televisyen (TV) swasta di Taiwan, baru-baru ini.', 'KEBIASAANNYA, pasangan pengantin baru akan bercuti untuk berbulan madu dan menghabiskan masa bersama.', 'Keempat-empat mereka mendakwa tidak mengetahui bahawa beruang madu merupakan spesis hidupan liar yang dilindungi sepenuhnya bukan sahaja di Indonesia, tetapi seluruh dunia.', 'Ketika berlaku pergelutan, suspek wanita yang merupakan madu mangsa bertindak merampas beg tangan mangsa yang mengandungi tiga buah telefon bimbit, kad ATM dan wang tunai sebanyak RM250, katanya kepada pemberita di sini hari ini.', 'Kos bagi mewujudkan makmal tersebut kira-kira RM172,000 termasuk pembelian pelbagai peralatan dan mampu memproses 50 kilogram madu sehari.', 'KOTA TINGGI : Kementerian Kesihatan sedang menyiasataduan produk madu yang diimport dari Australia yang didakwa mengandungikandungan beracun dan bahan toksin yang tinggi, yang berada di pasaran tempatan.', 'KUALA LUMPUR : Kegembiraan seorang isteri untuk berbulan madu di rantau ini nampaknya tergencat apabila suaminya, Jonathan Kemp, 27, menghilangkan diri selepas berlaku pertengkaran sesama mereka di sini, semalam.', 'KUALA LUMPUR : Sepasang suami isteri sepatutnya pulang ke tanah air hari ini setelah menikmati percutian bulan madu mereka di Beijing.', 'KUALA NERUS : Penubuhan Makmal Kualiti Madu (MKM) pertama di Pantai Timur di Universiti Malaysia Terengganu (UMT) mampu menyelesaikan masalah lambakan madu tiruan yang berleluasa di pasaran, kata seorang anggota Exco negeri.', 'KUANTAN : Jika sebelum ini durian Musang King, nangka madu dan manggis dari negeri ini berjaya menembusi pasaran antarabangsa sehingga ke China dan Eropah, kali ini tembikai dari Ubai, Pekan pula memasuki pasaran Dubai.', 'Kuih makmur, kuih semprit, dan kuih emping jagung disaluti madu menjadi satu pilihan wajib untuk dihidangkan dalam balang atau bekas kuih.', 'Lebah kelulut itu boleh menghasilkan madu yang mempunyai dua hingga empat kali ganda nilai anti oksidan yang baik untuk tubuh badan manusia, katanya.', 'Makanan yang dibekalkan ialah seperti bijirin, susu rendah lemak, buah kismis, kurma, madu dan minuman protein, katanya.', 'Makmal yang pertama kali dibangunkan itu menawarkan perkhidmatan pengeringan madu kelulut, pembungkusan dan analisis ringkas penentuan kualliti madu kelulut.', 'Malah kami turut dihadiahkan pakej bulan madu tiga hari dua malam dan sijil simpanan Premium Bank Simpanan Nasional bernilai RM1,000 untuk simpanan masa depan, katanya yang berasal dari Kampung Gong Tok Nasek, Kuala Terengganu.', 'Malah tidak kurang juga yang arif mengenai beberapa sumber makanan seperti coklat gelap, badam, tiram, strawberi, asparagus, madu dan lain-lain yang berupaya meningkatkan ghairah seksual.', 'Mat Hussin Daud, 60 tahun, bangkit mengusahakan 100 batang pokok nangka madu di atas tanah seluas 0.', 'Menurut beliau, bersengat boleh diibaratkan seperti seekor lebah yang memberi manfaat melalui madu yang berkhasiat dan sentiasa patuh dengan sunatullah seekor lebah kepada Penciptanya agar kehidupannya mendapat berkat.', 'Menurut Misbah pula, minat itu timbul apabila Kerajaan Negeri Terengganu menerusi Yayasan DiRaja Sultan Mizan (YDSM) dan Pusat Sains Dan Kreativiti Terengganu mengadakan seminar mengenai madu kelulut sekitar tahun 2012.', 'Menurut Noh, misi kemanusiaan yang disertai dia dan Mizz Nina saling tidak ubah seperti bulan madu buat mereka berdua.', 'Menurut sumber, pasangan itu bakal berbulan madu pada 28 Disember ini.', 'Menurutnya, madu kelulut juga mengandungi karbohidrat yang tinggi, asid amino, vitamin dan asid organik yang amat berkhasiat untuk tubuh badan manusia.', 'Mereka adalah pengusaha ternakan madu lebah kelulut, penternak kambing moden dan petani fertigasi rock melon yang berkongsi pengalaman serta cabaran masing-masing dalam membina empayar perusahaan masing-masing.', 'Mereka bercadang untuk berbulan madu di Istanbul sebelum bergerak ke Mekah.', 'Mereka yang berkahwin pada 27 November lalu dikatakan sudah berangkat untuk meraikan bulan madu di beberapa buah negara Eropah.', 'Misi yang ditaja RH Bee Farms Sdn Bhd, pengeluar madu lebah kelulut terbesar di Sarawak dan anak syarikat Rimbunan Hijau Group itu diadakan bersempena Pameran Pertanian, Hortikultur dan Agro Pelancongan Malaysia (MAHA) 2016 pada 24 November hingga 4 Disember ini di Serdang, Selangor.', 'Najib, yang juga presiden UMNO dan pengerusi Barisan Nasional (BN), menyifatkan manifesto pakatan pembangkang itu penuh dengan janji manis umpama madu tetapi sebenarnya racun berbisa.', 'Namun, Adib memberitahu mereka masih melalui fasa berbulan madu untuk hidup berdua.', 'Namun, selepas 12 hari, mereka menemui seekor beruang madu yang terjerat.', 'PASANGAN popular, Priyanka Chopra dan suaminya Nick Jonas memilih Switzerland sebagai lokasi berbulan madu mereka.', 'PELAKON muda, Datin Elvina Mohamad belum merancang untuk berbulan madu selepas selamat diijabkabulkan semalam.', 'Pilihan lain seperti yogurt smoothies atau susu vanila/ susu bersama madu untuk memstikan perut sentiasa kenyang dan kekal hidrasi sepanjang hari.', 'Pok Sabri berkata beliau akan mempromosikan penggunaan madu lebah kelulut serta MAHA sepanjang misi itu dan dijadulkan menamatkan Kelulut Walk di Taman Ekspo Pertanian Malaysia Serdang (MAEPS), tempat berlangsungnya MAHA 2016.', 'Pok Sabri yang pernah aktif dalam pelbagai aktiviti sukan lasak berkata beliau mula mengamalkan madu lebah kelulut selepas diperkenalkan oleh seorang rakannya yang juga pengusaha madu lebah itu.', 'Prachaub Kanpecth, yang selalu mengumpul bahan buangan dan madu hutan, didakwa pada Selasa kerana memiliki dan berniat untuk membekalkan bahan terlarang itu, lapor AFP .', 'RIAU : Empat lelaki berdepan hukuman penjara atas kesalahan menangkap tiga ekor beruang madu dan memasaknya menjadi rendang.', 'Rita memuat naik gambar pertemuannya bersama madu kepada aktres Izreen Azminda, Intan Liana Saleh.', 'Rosyam sebelum ini pernah menceburi bidang pertanian menerusi projek tanaman nangka madu J33 dan durian belanda.', 'Saravanan bertekad untuk meluaskan perusahaan madu kelulut di kawasan tempat tinggalnya itu.', 'Saravanan memulakan perusahaan madu kelulut ini dengan modal RM35,000 untuk membeli 10 sarang lebah dan peralatan untuk memproses madu.', 'Saya dan isteri pertama sanggup terima madu kami sekiranya dia menjalankan tanggungjawab dengan baik, ? katanya.', 'Saya dapati madu kelulut ini mampu menambah tenaga di samping meningkatkan tahap kesihatan," jelasnya.', 'Sebahagian mereka yang ditemui Astro AWANI di sini, kini mula beralih kepada penternakan kelulut yang menghasilkan madu sebagai sumber pendapatan baharu dan sampingan yang menguntungkan.', 'Selain berbuah sepanjang masa serta mengeluarkan hasil cepat, tanaman nangka madu mudah diusahakan dan dijaga.', 'Selain terpaksa berbulan madu di penjara, pasangan itu juga tidak dibenarkan melawat bayi mereka yang kini ditempatkan di rumah kebajikan.', 'SELEPAS kejayaan Fantasia Bulan Madu menerusi album Mentari Merak Di ufuk Timur, kumpulan Search mencipta satu lagi kejayaan menerusi lagu Isabella.', 'Selepas mesyuarat tersebut, Najib yang juga Menteri Kewangan diperkenalkan kepada program Pusat Inovasi Komuniti Kubu Gajah mengenai pengeluaran madu yang berkualiti tinggi melalui aplikasi Meliponiculture.', 'Sementara itu beliau berkata, kementeriannya akan bekerjasama dengan Kumupulan Rimbunan Hijau dan pengusaha-pengusaha industri ternakan lebah kelulut untuk mempromosikan penggunaan madu lebah kelulut dalam kalangan rakyat Malaysia.', 'Sementara itu, Mohd Soffi berkata TKPM yang terkenal dengan pengeluaran nangka madu dan rebung madu itu juga sehingga kini telah menghasilkan pendapatan RM82 juta dalam tempoh 14 tahun sejak mula di tubuhkan pada tahun 2004.', 'Sementara itu, Penolong Pegawai Pertanian Jabatan Pertanian Gua Musang, Alias Muhammad berkata, tanaman nangka madu sememangnya berpotensi untuk dikomersialkan Katanya, sehingga kini kira-kira 50 pekebun mengusahakan nangka madu secara kecil-kecilan.', 'Seorang pelancong tempatan dari Tawau, Jefri Jajih pula berbulan madu bersama isterinya di Pulau Mabul kerana merasakan keadaaan di pulau itu adalah selamat.', 'SEOUL : Hawaii merupakan destinasi paling popular untuk berbulan madu tahun lepas, diikuti pulau peranginan Phuket di Thailand, lapor agensi berita Yonhap pada Isnin memetik kenyataan agensi pelancongan terbesar di Korea Selatan.', 'Seramai 260 penternak lebah kelulut yang berdaftar boleh mendapatkan sijil pengesahan keaslian madu lebah kelulut yang dikeluarkan oleh makmal ini dan dengan adanya sijil pengesahan, pelanggan boleh membezakan madu yang asli atau sebaliknya, sekali gus mengelak daripada ditipu peniaga yang tidak jujur.\r', 'Sudah pergi berbulan madu di Taiwan, kenapa ragu-ragu? Jika benarlah Joy tidak pandai memberikan kerjasama menyelesaikan masalah, dia sebagai suami perlu bertindak lebih matang.\r', 'SUNGAI PETANI ": Jentera pilihan raya UMNO dan Barisan Nasional (BN) tidak boleh terus berada dalam suasana berbulan madu jika mahu BN terus mentadbir negeri dan negara, kata Anggota Majlis Tertinggi UMNO Datuk Seri Tajuddin Abdul Rahman.', 'Sunye dan bakal suami dikhabarkan memilih Maldives sebagai destinasi berbulan madu sebelum berpindah ke Kanada untuk memulakan kehidupan bersama.', 'Terbaharu, Rita mencuri perhatian ramai apabila telah memuat naik sekeping gambar yang menunjukkan dia bersama-sama madu Izreen iaitu Intan Liyana, selain pelakon Nazia Mustapha dan seorang lagi rakan mereka.', 'Untuk peringkat awal ini Mardi akan mengenakan caj RM5 sahaja untuk sekilogram madu kelulut yang mahu diproses.', 'USAHAWAN kosmetik yang kini bergelar penyanyi, Datuk Seri Aliff Syukri mengakui dia mengambil kesempatan terhadap madu kepada aktres Izreen Azminda, Intan Liana Saleh.', 'Lebih manis selepas perkahwinan Arpita dan suami, mereka bakal berbulan madu di sebuah lokasi yang masih dirashiakan yang turut dirancang sendiri oleh Salman khas buat mereka berdua.', 'Di sini Astro Gempak ingin berkongsi foto-foto menarik berbulan madu Abby dan Jai yang diambil menerusi Instagram mereka.', 'Ramai yang tahu bahawa pasangan popular ini sedang berbulan madu setelah sibuk dengan tugasan seni masing-masing.', 'Walaupun bukan seperti madu dan racun, tetapi pilihan antara Jokowi dan Prawobo barangkali boleh dikaitkan dengan tawaran manis dan pahit atau baru dan lama.', '[BACA: Ya, Memang Saya Ambil Kesempatan Terhadap Isteri Pertama Ude" - Aliff Syukri] Tambah penyanyi lagu Abang Nak Tegur itu, dia mengambil madu kepada aktres Izreen Azminda itu sebagai model untuk menambahkan pendapatan Intan yang merupakan ibu kepada dua orang anak.', 'Ketua Pengarah Kesihatan Datuk Dr Noor Hisham Abdullah berkata penarikan balik produk madu itu dibuat Syarikat Evergreen Life Limited di New Zealand kerana terdapat maklumat kemungkinan ada bahan tidak dibenarkan iaitu dihydroxyacetone dan methylglyoxal yang digunakan dalam pemprosesannya.', 'Jadi tahun ni takde biskut raya corn flakes madu ke? Ermmm ? Lilian (@leyli_lily) July 13, 2014 Herm herm bnyk ni kena boikot.', 'Kerajaan negeri turut menggalakkan program berbulan madu bagi pasangan yang telah lama berkahwin bagi memperkukuh hubungan suami isteri.', 'KUALA TERENGGANU : Satu kawasan di Marang akan dijadikan pusat ternakan lebah madu dalam masa terdekat oleh Kerajaan Negeri Terengganu dalam usaha membangunkan tanah-tanah terbiar di negeri itu.', 'Selepas majlis resepsi pada Jumaat, pasangan ini bakal berlepas ke Rom, Milan, Venice, Frankfurt dan Amsterdam untuk berbulan madu selama dua minggu mulai 16 Disember.', '?Kalau kerajaan BN itu umpama bunga, maka warga pekerja ibarat tangkainya, justeru jangan sesekali terpesona dan tergoda dan menghirup madu di ?bunga? lain.', 'menikmati madu dari kelopak Barisan Nasional yang tulen, lagi menjadi penawarnya, Usahlah memilih ?mata yang lebam?, kelak tersesat di hujung jalan.', 'Kami tidak mempunyai apa-apa perancangan untuk pergi berbulan madu tapi ya Baitulmuqaddis.', 'Alfrits sebelum ini merancang untuk membawa Devita berbulan madu di Bandung sebaik sahaja penyakitnya sembuh.', 'Ada yang berbulan madu dengan menunaikan umrah.', ' Sudah tentunya detik persekolahan adalahyang ditunggu-tunggu bagi pelajar sekolah untuk bertemu kembali bersama rakan-rakan lain selepas 42 hari bercuti, namun ia mungkin sedikit sukar bagi ibu bapa apabila keseronokan dan tempoh bulan madu telah berakhir dan realiti awal pagi menanti anda kembali.', 'Izzar juga telah merancang untuk berbulan madu ke dua buah lokasi iaitu di dalam dan luar negara.', 'Ummi telah memilih Sabah dan Switzerland dan akan menikmati bulan madu mereka selepas selesai majlis respesi ketiga yang akan berlangsung di Dewan Banquet FELDA pada 12 Ogos.', 'Hutan Simpan Ampang juga menjadi habitat kepada lebih 100 spesies unggas, flora dan fauna termasuk tapir, beruang madu dan harimau bintang.', 'Bulan madu pasangan ini juga turut mendapat tajaan syarikat DCL.', 'u2028 ?Dia sengaja jual kek, produk-produk yang ada seperti mana yang saya bagi tahu, susu, madu yang dicampur dengan ganja dan dia pun telah digelar sebagai ?doktor ganja? dalam laman web.', 'Dalam serbuan itu polis turut merampas 18 batang pokok ganja, empat kilogram tumbuhan dan serbuk ganja, 72 kilogram benih ganja, empat liter minyak ganja, 70 keping coklat ganja, 66 botol madu ganja, 21 botol majerin ganja, sembilan botol susu ganja, dua biji kek ganja serta pelbagai peralatan menghisap ganja.', 'Selesai majlis resepsi, pasangan ini bakal berlepas ke Eropah untuk berbulan madu selama dua minggu.', '?Saya bersedia untuk berjumpa dengan isteri baru Bade iaitu madu saya secara terancang mahupun kebetulan, saya serahkan kepada suami saya bagaimana cara pertemuan itu.', '?Kalau kami ada masalah rumah tangga takkanlah kami akan bercuti bersama ke luar negara? Ia seperti percutian bulan madu sekali gus mengeratkan lagi ikatan rumah tangga sedia ada,? luahnya kesal.', 'Dulu dia tuduh Anwar macam-macam, sekarang dia kata Anwar dizalimi disebabkan perbezaan pendapat politik dan tidak mahu Anwar di gelanggang politik, " Malah semasa ucapannya itu, Ahmad Zahid turut menyeru prinsip habis madu sepah dibuang tidak diamalkan di kalangan pemimpin akan datang.', 'Sementara itu, Shamsul Bahri yang juga Ketua Kumpulan Penyelidik Lebah dan Kelulut (SIG) UMT berkata pihaknya menerima banyak laporan mengenai aktiviti penjualan madu yang dicampur dengan air gula oleh segelintir penternak yang boleh membahayakan kesihatan pengguna.', 'Beliau berkata kegiatan tersebut boleh membunuh industri madu kelulut yang semakin berkembang di negara ini.', 'BACA: Tanam janin di tepi sawah, tiga individu dijejaki ARTIKEL BERKAITAN: Bayi dihurung semut dan masih bertali pusat ditemui tepi jalan ARTIKEL BERKAITAN: Bayi dibuang di tong sampah dengan mulut disumbat tisu diselamatkan ARTIKEL BERKAITAN: Pasangan kekasih bersalah buang bayi berbulan madu dalam penjara Semalam, media melaporkan perbuatan mencurigakan lelaki berkenaan di kawasan sawah di Bukit Rambai di sini kira-kira 8 pagi semalam disedari penduduk kampung sebelum membuat laporan polis.', 'KEMAMAN : Pihak swasta digalak membuka makmal atau pusat pemprosesan madu kelulut bagi menghasilkan madu berkualiti dan mampu menjana keuntungan, kata Menteri Pertanian dan Industri Asas Tani Datuk Seri Ahmad Shabery Cheek.', 'kebetulan permintaan tinggi, ia dibimbangi berlaku ketidakcukupan tenaga kerja dan melambatkan urusan pengusaha dalam mendapatkan madu yang telah diproses, katanya dalam sidang media selepas merasmikan Makmal Pemprosesan Madu Kelulut Kemaman, hari ini.', 'Di ruangan lidah pengarang The Jakarta Globe", ada menyebut mengenai kisah bulan madu Jokowi yang mula tamat sebaik sahaja angkat sumpah berakhir.', 'Tidak menghalang Kaka untuk meneruskan karier seni, Adam turut berkata, dia akan membawa isterinya berbulan madu dengan mendaki Annapurna di Nepal, awal tahun depan.', 'Pelakon Sasqia Dahuri yang sah menjadi isteri ahli perniagaan hartanah dari Singapura, Muhammad Asyraff Khan Baharudin Khan, malam tadi memaklumkan, hasrat mereka berbulan madu di London terpaksa ditunda.', 'Sebagai kafe yang mengutamakan status halal, kami tidak menggunakan mirin, sake dan cuka alkohol tetapi di tukar kepada garam, gula, madu, cuka makan, gula kabung dan kicap manis. ', 'Suspek, yang bekerja sebagai penjual madu ditahan di Kelantan disyaki menerima arahan daripada Akel Zainal, seorang rakyat Malaysia yang menyertai Daesh di Syria.', 'Batu-bata dilekat menggunakan bancuhan campuran putih telur itik, pasir, kapur, gula merah dan madu, tiada besi atau paku digunakan', 'Kalau kita berjalan dengan membawa sifat seperti lebah iaitu mengutip madu atau yang manis, maka kita dapat belajar banyak perkara kebaikan," katanya.', 'Pasangan yang mula berkenalan sejak lapan tahun lalu itu memilih Bali, Indonesia sebagai destinasi bulan madu seterusnya ke Australia atau Eropah sebelum penghujung 2019.', 'MOHD Derus (kiri) melihat kerosakan pokok nangka madu jenis J33 di kebunnya yang musnah angkara gajah di TKPM, Kota Perdana', 'Shuib atau nama sebenarnya Shahmira Muhamad berkata, cabaran terbabit dilakukan bagi mendalami ilmu berkaitan serangga berkenaan memandangkan dia menceburi perniagaan madu.', 'Bagi peminat madu lebah, kelebihan makanan semula jadi ini memang diperakui bukan sahaja mengenyangkan dan menyihatkan tubuh badan, malah mampu menyembuh pelbagai penyakit.', 'Wartawan BH memenangi Anugerah Kewartawanan Kesihatan Terbaik 2018 (Akhbar Bahasa Melayu) menerusi rencana bertajuk Madu Diabetes.', 'Apabila saya memaklumkan mahu berniaga madu, dia memuji usaha saya.', 'Saya berhasrat menjadikan syarikat ini peneraju industri madu terbesar di Malaysia," katanya yang kini memiliki lebih 20 stokis di seluruh negara.', "Beliau berkata, pada masa ini, syarikat itu mempelbagaikan produknya kepada makanan kesihatan menerusi pelancaran produk terkini, jus buah-buahan campuran 'JustGreat', mengandungi koktel durian belanda, mangga, limau calamansi dan pilihan madu semula jadi.", 'Susulan pengakuan Datin Seri Umie Aida yang mengizinkan suami berkahwin lagi, kakaknya, Erma Fatima berkongsi pengalaman dirinya yang pernah hidup bermadu.', ' Calon PKR bagi Pilihan Raya Kecil (PRK) Sungai Kandis, Mohd Zawawi Ahmad Mughni, menjadikan amalan meminum madu kelulut sebagai tonik tenaga ketika kempen memasuki hari kedua.', 'Jadi, soal bercerai atau perkahwinan tidak diterima madu anak saya, biarlah menantu saya yang uruskan kerana kami orang miskin," katanya.', "Aduhai baru 11 tahun dah jadi madu'", 'Sudah dua tahun saya mengamalkan diet sedemikian untuk sahur. Bergantung kepada kitalah sebanyak mana mahu meletakkan kurma. Kadang-kadang, saya campur oat dengan madu.', 'Nusaraya Resources Sdn Bhd, syarikat pengeluar minuman kesihatan milik Bumiputera menyumbangkan 1,001 kotak produk minuman madu Roselle SiHulk kepada 1,001 pesakit barah.', 'Madu lebah memiliki semua khasiat yang diperlukan badan manusia seperti enzim, vitamin, mineral, air dan pinocembrin.', 'Begitu luahan isteri pemuzik, Muhammad Radzi Abdul Wahid, Intan Liana Saleh, 32, yang kini menjadi madu kepada pelakon, Izreen Azminda.', 'Saya reda. Saya terima hakikat diri ini bermadu.', 'Pasangan itu ditahan ketika sedang bermadu asmara di dalam sebuah kereta di Arena Merdeka, Dungun, pada 8 April lalu.', 'Tambah Umie, dia tidak tahu punca dirinya bertindak demikian , apatah lagi tidak ada isteri yang suka hidup bermadu.', 'Walaupun tidak lagi hidup bermadu selepas Ude menceraikan isteri pertama, Intan Liana Saleh, Mac lalu, namun api kebencian netizen masih tidak mampu dipadamkan sepenuhnya.', "Biarpun menerima banyak mesej 'lamaran' daripada sesetengah wanita yang ingin menjadi madunya, Sheila, 33, memilih untuk tidak melenting kerana cukup mengenali hati budi suami.", 'Bagi wanita, rata-rata pasti mengakui hidup bermadu umpama mimpi ngeri yang seboleh mungkin cuba dielak.', 'Bagi wanita yang bermadu atau suaminya dirampas orang lain, jangan lagi dikenang perkara yang lalu.', 'Beliau yang berkongsi pengalamannya bermadu sejak lapan tahun lalu berkata, wanita tidak sepatutnya menolak hukum disyariatkan Allah SWT sehingga meletakkan kesalahan atau memandang serong kepada lelaki yang memilih untuk berpoligami.', 'Penyanyi, Adira Suhaimi, memilih untuk memekakkan telinga dan berdiam diri walaupun di luar sana masih banyak pihak mengecam keputusannya untuk hidup bermadu.', 'Sebelum ini, Izreen menjadi perhatian ramai apabila Intan Liana Saleh, 32, yang juga isteri pertama suami aktres itu, Muhammad Radzi Abdul Wahid atau Ude, 29, membuat pengakuan rela hidup bermadu dengan pelakon itu.', 'Dua suspek wanita yang bermadu itu pula akan menyamar sebagai jururawat lengkap memakai pakaian seragam dan masuk ke rumah mangsa berpura-pura melakukan pemeriksaan kesihatan ke atas mangsa sebelum melarikan barang kemas serta wang tunai.', 'Katanya, dua suspek wanita masing-masing berusia 36 dan 25 adalah bermadu manakala seorang lagi ahli sindiket ialah lelaki berusia 30 tahun.', ' Pelakon, Ruhainies, tidak berminat untuk hidup bermadu atau menjalinkan hubungan cinta dengan suami orang.', 'AJAK menolak dengan baik kebenaran diberikan isterinya, Nur Afifah, yang rela dirinya dimadukan.', 'Adira mengaku bermadu', 'Hampir tiada wanita sanggup berkongsi suami kerana menganggap hidup bermadu adalah sesuatu yang menakutkan.', 'Nikmat bermadu empat', 'Cabaran hidup bermadu bukan senang ', 'Zarina Zainuddin tunda tarikh perkenal suami, madu', 'Hidup bermadu ujian hebat saya dan Aida - Erma Fatima', 'Kecewa selepas mengetahui dirinya dimadukan, seorang wanita bertindak keluar rumah membawa bersama anak sebelum mereka menghilangkan diri, Jumaat lalu.', 'Katanya, di kawasan berkenaan mempunyai ibu tunggal dan isteri yang dimadukan, justeru mereka ada masalah tersendiri.', 'MADU ialah makanan yang sangat bernilai yang digunakan sejak zaman purba sebagai sumber makanan tenaga yang penting.', 'Dalam al-Quran madu disebut secara spesifik sebagai makanan ahli syurga dan di dalamnya mengandungi penyembuh penyakit untuk manusia.', 'Madu digunakan secara meluas dalam perubatan tradisional sebagai rawatan alternatif untuk pelbagai penyakit, pemakanan, dan kesihatan manusia. ', 'Menurut Suruhanjaya Alimentarius Kodeks (1989), madu didefinisikan sebagai pemanis semula jadi dihasilkan lebah madu dari sari bunga atau nektar atau dari rembesan bahagian hidup tumbuhan, atau perkumuhan tumbuhan, dikumpulkan oleh lebah madu dan diubah, serta digabungkan dengan bahan spesifik dalam lebah madu sehingga matang.', 'Secara umum, madu boleh dikategorikan kepada dua jenis iaitu madu monoflora, dan madu poliflora.', 'Madu monoflora ialah madu yang dihasilkan dari nektar dari sejenis bunga saja. ', 'Madu poliflora atau dikenali sebagai madu multiflora ialah madu yang dihasilkan lebah madu yang mengambil nektar dari pelbagai jenis bunga. ', 'Sifat fizikal madu seperti warna, aroma, rasa, dan ketumpatan madu serta komposisi kimia sesuatu madu bergantung kepada jenis nektar bunga yang dihisap lebah madu,', 'Komposisi utama madu ialah gula, iaitu antara 95 hingga 97 peratus berat kering madu. Gula utama dalam madu ialah daripada fruktosa dan glukosa.', 'Vitamin yang terkandung dalam madu termasuk vitamin B, vitamin A, dan vitamin C, manakala mineral yang biasa terkandung dalam madu ialah kalium, magnesium, kalsium, natrium, sulfur, besi dan fosfat.', 'Dalam madu ada pelbagai enzim yang tidak terdapat dalam gula putih. ', 'Kajian saintifik membuktikan madu mempunyai kesan antimikrob, antikeradangan, serta antipengoksida semula jadi.', 'Kewujudan madu tiruan dalam pasaran tempatan diakui menjejaskan lebih 50 peratus pendapatan peniaga madu asli apabila produk mereka turut diragui orang ramai.', 'Bagi pakar madu seperti ayah saya, mereka tahu ciri madu asli walaupun ada cubaan menipu dengan mencampurkan dengan gula melaka.', 'Kami mendapat bekalan madu tualang ini dari Indonesia sebelum diuji di makmal di Universiti Kebangsaan Malaysia (UKM) di Bangi, bagi mengenal pasti keasliannya.', 'Kita akan membuat pemantauan terhadap produk madu yang diusahakan pengusaha negeri ini termasuk penelitian terhadap aspek kandungan produk.', 'Selain itu, jika dicampur air mineral, madu tualang tiruan akan menjadi jernih, manakala yang asli menjadi keruh," katanya.', 'Katanya, hasil kajian dilakukan tiga pelajar berkenaan menemukan khasiat dan kebaikan campuran dua madu keluaran tempatan iaitu madu tualang dan madu kelulut yang boleh mencegah kanser dan merawat penyakit radikal berkaitan kardiovaskular.', 'Dari perut lebah keluar minuman (madu) yang bermacam-macam warnanya, di dalamnya terdapat ubat yang menyembuhkan bagi manusia", Surah An-Nahl:69', 'Madu merupakan sebuah cairan yang menyerupai sirup yang dihasilkan oleh lebah madu.', 'Madu memiliki rasa manis yang tidak sama dengan gula atau pemanis lainnya.', 'Madu dihasilkan dari jenis lebah, iaitu lebah liar dan lebah budidaya.', 'Ada berbagai jenis madu', 'Selain madu, peternak dapat memperoleh hasil lain dari lebah yang diternak berupa lilin lebah, tepung sari, dan royal jelly.', 'Di dalam madu terjadi unsur oksidasi yang membuat proses penguraian gula di dalam darah menjadi lebih mudah sehingga kadar gula tidak meningkat. ', 'Harus diakui bahwa selama ini pemasaran madu lebah di dalam negeri sering mengalami kesulitan.', 'Oleh itu ubat yang sesuai ialah dengan meminum madu.', 'Madu itu mestilah daripada tujuh jenis bunga.', 'Madu itu hendaklah diminum sesudu sebelum tidur." kata tabib. ', 'Walaupun hati kecil Rudjini sangat bersetuju, tetapi sekiranya dia berbuat begitu, pasti madu-madunya yang lain memusuhinya.', 'Madu alami umumnya terbuat dari "neetar" yaitu suatu cairan manis yang terdapat di dalam "mahkota bunga" yang biasa diserap oleh lebah yang kemudian disimpan di dalam sarang untuk diolah menjadi "bahan persediaan" makanan ', 'Madu lebah: ubat yang turun bersama wahyu', 'Dahsyatnya mukjizat madu: untuk kesehatan, kecantikan, dan kecerdasan', ' "Lebah madu akan mengeluarkan madu." Orang beriman ibarat lebah madu. ', 'Lihatlah botol madu yang ada di meja makan.', 'Pagi tadi mungkin kamu mengoleskan sedikit madu yang manis dan berwarna keemasan itu ke atas roti panggangmu, tanpa menaruh perhatian sedikit pun. ', 'Bermadu tak salah, kelakuan yang salah alahai', 'Adakah menjadi isteri kedua atau bermadu itu berdosa?', 'Laman ini dibina utk berkongsi kisah rumah tangga yg bermadu or poligami.', 'Bermadu itu sesuatu yang tidak ada faedahnya,malah ramai juga yg berfikir bahawa bermadu itu sekadar untuk menunjuk nunjuk bagus..', 'Dengan bermadu , saya belajar redha , saya belajar sabar , saya belajar jadi insan yang kuat .. Dengan bermadu juga , saya belajar erti sebuah perkongsian suka dan duka .', 'Hanya yang merasa tahu indahnya hidup bermaduu. ', 'Bermadu Itu Indah.. Wanita Ini Kongsi Pengalaman Manis Hidup Bermadu.. Memang Indah Kalau Macam ini..', 'BERMADU ? Fitrah wanita tidak boleh berkongsi, Hati akan sakit, cemburu akan menguasai diri. ITU HAKIKAT! Isteri Rasulullah pun cemburu. ', 'Siapa nak bermadu boleh hubungi adik ni.. Dia tak boleh mengandung tapi nak anak.. Kasihan dia...', 'Perempuan mana yang hendak dimadukan? ', 'Baru-baru ni saya dimadukan dengan seorang janda yang berstatus pengurus besar.', 'kenapa wanita cantik dimadukan..? kenapa wanita cantik diceraikan..? kenapa wanita cantik ditinggalkan...?', 'Menolak untuk dimadukan adalah haram apabila tiada alasan yang munasabah dan dapat diterima oleh syarak. Ini disebabkan oleh hadis yang menyebutkan : "Sekiranya mana-mana wanita meminta cerai daripada suaminya tanpa sebab yang kukuh, maka terhalanglah dia daripada mencium haruman syurga." (Hadis riwayat Abu...', 'Aku tak sanggup dimadukan...jadi aku rela mengundur diri kerana aku lelah dengan permainanmu']
len(input_sentences)

247

The ground truths are manually labeled sense clustering for each sentence, and the list length should be equal to the input_sentences length.

In [0]:
madu_gt=['1', '3', '1', '1', '1', '3', '3', '1', '3', '1', '1', '1', '1', '1', '3', '1', '3', '1', '3', '3', '1', '2', '2', '1', '1', '3', '2', '3', '3', '1', '3', '1', '1', '1', '3', '2', '1', '1', '3', '3', '3', '4', '1', '1', '1', '4', '1', '1', '2', '3', '3', '1', '1', '1', '1', '3', '1', '1', '1', '4', '1', '1', '3', '3', '4', '2', '1', '1', '3', '3', '1', '1', '1', '1', '1', '1', '3', '1', '1', '1', '1', '3', '3', '1', '1', '3', '3', '1', '1', '3', '4', '3', '3', '1', '1', '1', '1', '4', '2', '1', '1', '1', '2', '1', '1', '1', '3', '3', '1', '1', '1', '1', '3', '3', '1', '3', '3', '3', '2', '1', '2', '3', '3', '3', '1', '2', '1', '1', '3', '4', '3', '1', '1', '3', '3', '3', '3', '3', '3', '4', '3', '1', '1', '3', '2', '3', '1', '1', '1', '3', '1', '1', '3', '3', '3', '1', '1', '1', '1', '3', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '2', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2','2']
len(madu_gt)

247

The clustering functions are defined below:

In [0]:
def cluster_adj_words(sentencelist, targetword, algorithm, ncluster=3, adjacency=3, dir='both', randstate=100):
    #get neighboring words with adjacency=adjacency for all sentences
    #randstate=300
    adjacentwords=[]
    targetword=targetword.lower()
    for s in sentencelist:
        s=s.lower()
        l=s.split(' ')
        targetfound=False
        #search for word in reverse order to get first occurrence
        for x in reversed(range(len(l))):
            if re.search(targetword,l[x]):
                targetfound=True
                ctr=x
        if not targetfound:
            print("Target word not in sentence",s)
            continue
        else:
            #print("Getting adjacent words")
            #get index location for target word
            center=ctr
        #get words in adjacency
        r=[]
        tmp=''
        #get index sequence of what words need to be included
        for i in range(1,adjacency+1):
            if dir=='left' or dir=='both':
                r.append(center-i)
            elif dir=='right' or dir=='both':
                r.append(center+i)
        r=sorted(r)
        
        for index in r:
            if index>0 and index<len(l):
                tmp=tmp+' '+l[index]
        adjacentwords.append(tmp)
       
    #create count array
    Xtc=CountVectorizer().fit_transform(adjacentwords)
    Xtf=term_freq(Xtc)
    Xtfidf=TfidfVectorizer().fit_transform(adjacentwords)
    tc_cluster=k_means(Xtc.toarray(),ncluster, algorithm=algorithm, random_state=randstate)
    tf_cluster=k_means(Xtf,ncluster, algorithm=algorithm, random_state=randstate)
    tfidf_cluster=k_means(Xtfidf,ncluster, algorithm=algorithm, random_state=randstate)
    return tc_cluster, tf_cluster, tfidf_cluster
  
def cluster_all(sentencelist, targetword, ncluster=3):
    randstate=300
    Xtc=CountVectorizer().fit_transform(sentencelist)
    Xtf=term_freq(Xtc)
    Xtfidf=TfidfVectorizer().fit_transform(sentencelist)
    tc_cluster=k_means(Xtc.toarray(),ncluster, random_state=randstate)
    tf_cluster=k_means(Xtf,ncluster, random_state=randstate)
    tfidf_cluster=k_means(Xtfidf,ncluster, random_state=randstate)
    return tc_cluster, tf_cluster, tfidf_cluster

In [0]:
def cluster_sentences(sentencelist, targetword, input_ground_truth):
  #cluster with all words, input is sentence list
  #will clean up sentence and create pos_list and stem_list
  outstr=''
  tokenstype = ['TOKENS=WORDS', 'TOKENS=STEMMED WORDS', 'TOKENS=POS' ]
  title = ['USING TC:','USING TF:','USING TFIDF']
  
  #check that sentencelist and ground_truth are the same length
  if len(sentencelist)==len(input_ground_truth):
    print("sentence list and ground truths are the same length. Good to go!")
  else:
    print("ground truth and sentence list lengths mismatched!")
    return

  #filter list of sentences with targetword in original form
  tmp=sentencelist.copy()
  test_sentences=[]
  ground_truth=[]

  for i in range(len(tmp)):
    sentence=tmp[i].lower()
    if find_target(sentence, targetword):
      test_sentences.append(sentence)
      ground_truth.append(input_ground_truth[i])
  
  #define ground truth as number of clusters in the filtered ground_truth list
  nclust=len(set(ground_truth))
  
  #print out counts per sense
  print("##number of sentences per sense:")
  for sense in set(ground_truth):
    print("##sense:",sense,"=",ground_truth.count(sense))
  
  #create stemmed and pos versions
  print("creating pos and stem list")
  pos_list=[]
  for sentence in test_sentences:
    sentence=sentence.lower()
    pos1=[y for x,y in pos.predict(sentence)]
    pos1[[x for x,y in pos.predict(sentence)].index(targetword)]=targetword
    pos1=' '.join(pos1)
    pos_list.append(pos1)
  
  stem_list=[]
  for sentence in test_sentences:
    slist=list(sentence.lower().split(' '))
    idx=slist.index(targetword)
    left=' '.join(slist[:idx])
    right=' '.join(slist[idx+1:])
    try:#may fail if targetword is first word
      stem1=stemmer.stem(left)
    except:
      stem1=''
    try:
      stem2=stemmer.stem(right)
    except:
      stem2=''
    stemmed=stem1+" "+targetword+" "+stem2
    stem_list.append(stemmed)
  
  k=0
  for inlist in [test_sentences, stem_list, pos_list]:
    tokens=tokenstype[k]
    print("TOKENS="+tokens)
    k+=1
    #test and tabulate adj_rand score for kmeans clustering results
    direction=['left','right','both']
    algo=['full']
    adj=range(1,5)
    
    for a in algo:
      #print("kmeans algorithm, adjacency, number of clusters, direction, adj_rand_score\n")
      for d in direction:
        for n in adj:
          c1, c2, c3=cluster_adj_words(inlist, targetword, a, nclust, n, d)
          i=0
          for clust in [c1,c2,c3]:
            r=metrics.adjusted_rand_score(ground_truth, clust[1])
            txt=tokens+':'+title[i]+':adj='+str(n)+':nclust='+str(nclust)+':dir='+d+':score='+"{:.2f}".format(r)
            print(txt)
            outstr=outstr+txt+'\n'
            i+=1
            
    c1, c2, c3 = cluster_all(inlist, targetword, nclust)
    
    i=0
    for clust in [c1,c2,c3]:
            r=metrics.adjusted_rand_score(ground_truth, clust[1])
            txt=tokens+':'+title[i]+':adj= all tokens:nclust='+str(nclust)+':dir=both :score='+"{:.2f}".format(r)
            print(txt)
            outstr=outstr+txt+'\n'
            i+=1
  return outstr

Example of the full code execution:

In [0]:
test=cluster_sentences(input_sentences, 'madu', madu_gt)

Importing pickle file for another set of sentences and ground truths:

In [0]:
files=[path for path in pathlib.Path('gdrive/My Drive/Colab notebooks/clustering data').glob('*.pkl')]
for file in files:
  print(file)
  with open(file,'rb') as fh:
    sentencepkl=pickle.load(fh)
  targetword=sentencepkl[0]
  input_sentences=sentencepkl[1]
  input_gt=sentencepkl[2]
  outtxt=cluster_sentences(input_sentences, targetword, input_gt)
  with open('gdrive/My Drive/Colab notebooks/clustering data/outputs/'+file.stem+'.txt','w') as fh:
    fh.write(outtxt)

In [0]:
def get_representations(sentencelist, targetword, input_ground_truth):
  #return word, pos and stemmed word representations
  outstr=''
  tokenstype = ['TOKENS=WORDS', 'TOKENS=STEMMED WORDS', 'TOKENS=POS' ]
  title = ['USING TC:','USING TF:','USING TFIDF']
  
  #check that sentencelist and ground_truth are the same length
  if len(sentencelist)==len(input_ground_truth):
    print("sentence list and ground truths are the same length. Good to go!")
  else:
    print("ground truth and sentence list lengths mismatched!")
    return

  #filter list of sentences with targetword in original form
  tmp=sentencelist.copy()
  test_sentences=[]
  ground_truth=[]

  for i in range(len(tmp)):
    sentence=tmp[i].lower()
    if find_target(sentence, targetword):
      test_sentences.append(sentence)
      ground_truth.append(input_ground_truth[i])
  
  #define ground truth as number of clusters in the filtered ground_truth list
  nclust=len(set(ground_truth))
  
  #print out counts per sense
  print("##number of sentences per sense:")
  for sense in set(ground_truth):
    print("##sense:",sense,"=",ground_truth.count(sense))
  
  #create stemmed and pos versions
  print("creating pos and stem list")
  pos_list=[]
  for sentence in test_sentences:
    sentence=sentence.lower()
    pos1=[y for x,y in pos.predict(sentence)]
    pos1[[x for x,y in pos.predict(sentence)].index(targetword)]=targetword
    pos1=' '.join(pos1)
    pos_list.append(pos1)
  
  stem_list=[]
  for sentence in test_sentences:
    slist=list(sentence.lower().split(' '))
    idx=slist.index(targetword)
    left=' '.join(slist[:idx])
    right=' '.join(slist[idx+1:])
    try:#may fail if targetword is first word
      stem1=stemmer.stem(left)
    except:
      stem1=''
    try:
      stem2=stemmer.stem(right)
    except:
      stem2=''
    stemmed=stem1+" "+targetword+" "+stem2
    stem_list.append(stemmed)
  
  k=0
  representations=[test_sentences, stem_list, pos_list]
  
  return representations

files=[path for path in pathlib.Path('gdrive/My Drive/Colab notebooks/clustering data').glob('*.pkl')]

for file in files:
  print(file)
  with open(file,'rb') as fh:
    sentencepkl=pickle.load(fh)
  targetword=sentencepkl[0]
  input_sentences=sentencepkl[1]
  input_gt=sentencepkl[2]
  reps=get_representations(input_sentences, targetword, input_gt)
  outfile='gdrive/My Drive/Colab notebooks/clustering data/outputs/'+file.stem+'.pkl'
  with open(outfile,'wb') as fh:
    pickle.dump(reps, fh)


Some of the polysemy consistently scored low. The following is to check if the clustering results are consistent across different initialization, but different from ground truth.

In [0]:
files=[path for path in pathlib.Path('gdrive/My Drive/Colab notebooks/clustering data/outputs').glob('*.pkl')]
polysemy='bela'
tmp=[]
for f in files:
  if re.search(polysemy,f.stem):
    tmp.append(f)
for f in tmp:
  with open(f,'rb') as fh:
    sentencepkl=pickle.load(fh)
  tc1,tf1,tfidf1=cluster_adj_words(sentencepkl[1], polysemy, 'full', ncluster=2, adjacency=1, dir='both', randstate=100)
  tc2,tf2,tfidf2=cluster_adj_words(sentencepkl[1], polysemy, 'full', ncluster=2, adjacency=1, dir='both', randstate=200)
  tc3,tf3,tfidf3=cluster_adj_words(sentencepkl[1], polysemy, 'full', ncluster=2, adjacency=1, dir='both', randstate=300)
  
  #check ARI scores using one of clustering results as gt
  print(f.stem,":ARI for run1 vs run2:", metrics.adjusted_rand_score(tfidf2[1], tfidf1[1]))
  print(f.stem,":ARI for run1 vs run3:", metrics.adjusted_rand_score(tfidf3[1], tfidf1[1]))
  print(f.stem,":ARI for run2 vs run3:", metrics.adjusted_rand_score(tfidf3[1], tfidf2[1]))
  
  #write clustering result to output
  with open('gdrive/My Drive/Colab notebooks/clustering data/outputs/'+f.stem+'_cluster_results.pkl','wb') as fh:
    pickle.dump([tfidf1[1],tfidf2[1], tfidf3[1]],fh)

bela_1 :ARI for run1 vs run2: 1.0
bela_1 :ARI for run1 vs run3: 1.0
bela_1 :ARI for run2 vs run3: 1.0
bela_0 :ARI for run1 vs run2: 1.0
bela_0 :ARI for run1 vs run3: 1.0
bela_0 :ARI for run2 vs run3: 1.0
bela_2 :ARI for run1 vs run2: 1.0
bela_2 :ARI for run1 vs run3: 1.0
bela_2 :ARI for run2 vs run3: 1.0
